# Disease Classification by CNN

Based on [Disease Classification by CNN using MFCC](https://www.kaggle.com/gizemtanriver/disease-classification-by-cnn-using-mfcc) but implemented at the edge and pushed to [GitHub](www.github.com)

## Tensorflow version

This notebook will work the latest stable tensorflow 2 version (**CPU-only**). Change the `requirements.txt` file to `tensorflow-gpu` if you would like to work with the NVIDIA-compatible version of Tensorflow. Stay tuned for a version for AMD GPUs using Docker in the near future.

In [2]:
# Install required libraries
# Consider using a virtual environment to protect your device's state
!pip install -r ../requirements.txt

     |████████████████████████████████| 421.8 MB 80 kB/s  eta 0:00:015   |████▊                           | 61.9 MB 3.8 MB/s eta 0:01:36     |██████████▌                     | 138.0 MB 4.0 MB/s eta 0:01:11     |█████████████▌                  | 178.2 MB 3.4 MB/s eta 0:01:13     |██████████████                  | 184.7 MB 3.4 MB/s eta 0:01:11     |████████████████████▌           | 269.6 MB 3.8 MB/s eta 0:00:41
     |████████████████████████████████| 1.6 MB 3.8 MB/s eta 0:00:01
  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
Processing /home/carlos/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e/sklearn-0.0-py2.py3-none-any.whl
     |████████████████████████████████| 63 kB 2.4 MB/s eta 0:00:011
Processing /home/carlos/.cache/pip/wheels/cc/af/1a/498a24d0730ef484019e007bb9e8cef3ac00311a672c049a3e/absl_py-0.9.0-py3-none-any.whl
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
     |████████████████████████████████| 448 kB 4.1 MB/s eta 0:00:01
  

     |████████████████████████████████| 155 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 3.7 MB/s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.7.2-py3-none-any.whl size=1612883 sha256=d89e8deaab876f00e113771378feb32ed3e30c171fd7c38c9b028c4f4cebca77
  Stored in directory: /home/carlos/.cache/pip/wheels/18/9e/42/3224f85730f92fa2925f0b4fb6ef7f9c5431a64dfc77b95b39
  Created wheel for audioread: filename=audioread-2.1.8-py3-none-any.whl size=23091 sha256=f4a3d24cacbe8ae567b070145f7dde559fd39bd5d878ecaaf3e0e299c1e20539
  Stored in directory: /home/carlos/.cache/pip/wheels/cd/85/04/704a589ef8773d1ec1393c4b271a023f929d85ef5317b12d8c
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320720 sha256=3918e743705ce5e8ededf180105563c67c672b5d54292bc1e221941ec7cbbf24
  Stored in directory: /home/carlos/.cache/pip/wheels/a0/18/0a/8ad18a597d8333a142c9789338a96a6208f1198d290ece356c
Successfully built librosa audioread resampy


In [3]:
# Load various imports 
from datetime import datetime
from os import listdir
from os.path import isfile, join

import librosa
import librosa.display

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

Using TensorFlow backend.


In [7]:
mypath = "../data/respiratory_sound_database/audio_and_txt_files"
filenames = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.wav'))] 

In [8]:
p_id_in_file = [] # patient IDs corresponding to each file
for name in filenames:
    p_id_in_file.append(int(name[:3]))

p_id_in_file = np.array(p_id_in_file) 

# EXPLAIN WHAT MFCC IS!!

Using [LibROSA Demo](https://nbviewer.jupyter.org/github/librosa/librosa/blob/master/examples/LibROSA%20demo.ipynb) 
or 
[Read and Visualize Audio Files in Python](librosa module)(https://www.youtube.com/watch?v=vJ_WL9aYfNI)
Use them to deploy this part then put them below as resources

In [9]:
max_pad_len = 862 # to make the length of all MFCC equal

def extract_features(file_name):
    """
    This function takes in the path for an audio file as a string, loads it, and returns the MFCC
    of the audio"""
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', duration=20) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

**WORK IN PROGRESS** 